# Abstractive Text Summarization with Amazon Titan Using LangChain

## Why This Lab?

This lab demonstrates how to perform **abstractive text summarization** using **Amazon Titan** with the help of **LangChain**. Abstractive summarization is an essential task in natural language processing (NLP) where the goal is to generate a concise summary that captures the essential meaning of a document. This lab will guide you through leveraging **Amazon Titan's powerful language model** and **LangChain's flexible pipeline** to automate this process, making it easier to work with large amounts of text data.

---

---

## Tools & Technologies

In this lab, we will use:

- **Amazon Titan**: A state-of-the-art language model provided by Amazon Web Services, used for generating summaries and performing other text-based tasks.
- **LangChain**: A framework for developing applications using language models in a structured way.
- **AWS SDK (`boto3`)**: A Python SDK to interact with Amazon Web Services, particularly **Amazon Titan**.
- **Python**: The programming language used to implement and run the code in this lab.

---

## Step 1: Install Required Libraries

Before we start, we need to install and upgrade a few necessary libraries that will help us interact with **Amazon Titan** and **LangChain**. Run the following command in a code cell to install the required packages:
### Explanation:

- **langchain**: This library provides a framework to integrate language models and external tools (like AWS services) together.
- **langchain-core**: This package contains the core functionalities of LangChain, enabling model interactions.
- **langchain-community**: This package contains community-driven integrations for LangChain.
- **langchain-aws**: Provides the necessary integrations to interact with AWS services, including Amazon Titan.

⬇️Run this code 

In [1]:
pip install --upgrade langchain langchain-core langchain-community langchain-aws

Note: you may need to restart the kernel to use updated packages.


## Restart the Kernel

Once the libraries are installed and the environment is set up, it's often a good practice to **restart the kernel** to ensure that all changes take effect properly. This can help in clearing any cached variables or memory, ensuring that your environment is clean and up-to-date.

To restart the kernel:

1. Go to the **Kernel** menu in the Jupyter notebook.
2. Click on **Restart**.

### Why This Step?

- **Clears memory**: Restarting the kernel helps in clearing out old variables and functions that may cause conflicts or errors.
- **Ensures proper initialization**: It ensures that the libraries and environment variables set in Step 1 are properly loaded and available in the current session.


## Step 2: Set Up AWS Environment Variables

Before interacting with AWS services, we need to configure the **AWS environment variables**. These variables will allow us to specify settings like the **AWS region**, which are essential for interacting with services such as **Amazon Titan**.

### Why This Code?

This code sets the **AWS region** in which we will operate, allowing the application to interact with AWS resources like **Amazon Titan**.



In [1]:
import os

# Set your AWS environment variables
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # Update if needed


## Step 3: Initialize LangChain Client

Now that the environment is set up, we need to initialize the **LangChain client** to interact with **Amazon Titan**. This step involves specifying the **model ID**, **region**, and **AWS credentials** for accessing Amazon Titan and configuring its parameters.

### Why This Code?

This code initializes the **Amazon Titan model** through **LangChain** by specifying the model ID, region, and credentials. It also defines important model parameters such as the **maximum token count**, **temperature**, and **topP** for controlling the behavior of the model.

### Explanation of the Code:

- **`model_id="amazon.titan-text-express-v1"`**:  
  This specifies the **Amazon Titan** model that we want to use. The model ID uniquely identifies the specific variant of Amazon Titan. In this case, we're using the **text express version** of Amazon Titan (`amazon.titan-text-express-v1`).

- **`region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1")`**:  
  This line sets the **AWS region** in which the model is deployed. It retrieves the region from the environment variables (e.g., `us-east-1`). If the region is not set, it defaults to **`us-east-1`**. You can change the default region to match where your AWS resources are hosted.

- **`credentials_profile_name=os.environ.get("AWS_PROFILE", None)`**:  
  This specifies the **AWS profile** used for authentication. The `os.environ.get` function looks for a profile in the environment variables. If not provided, it defaults to **None** and uses the **default AWS profile**. You can configure different profiles for different AWS accounts or roles.

- **`model_kwargs`**:
  - **`maxTokenCount`**: Defines the **maximum number of tokens** (words or characters) the model can process in a single request. The higher the token count, the larger the document the model can handle in one go. In this example, the model is configured to handle up to **4096 tokens** per request.
  
  - **`temperature`**: Controls the **randomness** of the model's responses. A value of **0** means the output will be **deterministic** (i.e., always the same), while higher values (e.g., **0.7**) introduce more randomness and variability in the model’s output. In this case, **0** is used to get consistent results.
  
  - **`topP`**: This parameter defines the **diversity** of the responses generated by the model using **nucleus sampling**. A value of **1** means no filtering of tokens, allowing the model to generate the most diverse responses. A lower value (e.g., **0.9**) narrows down the selection to more probable tokens, reducing randomness.


In [2]:
from langchain_aws.llms.bedrock import BedrockLLM

llm = BedrockLLM(
    model_id="amazon.titan-text-express-v1",
    region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1"),
    credentials_profile_name=os.environ.get("AWS_PROFILE", None),  # ✅ Safe fallback
    model_kwargs={
        "maxTokenCount": 4096,
        "temperature": 0,
        "topP": 1,
    }
)

## Step 4: Load the Document for Summarization

In this step, we load the document that we want to summarize. The document could be in any format (e.g., text file, PDF), but for simplicity, we’ll use a plain text file.

### Why This Code?

This code reads the contents of a document (in this case, a text file) into a variable so that we can process the content and generate a summary based on it.
### Explanation of the Code:

- **`letter_path = "./letters/2022-letter.txt"`**:  
  This line specifies the **path** to the text file that contains the document you want to summarize. In this case, it points to a file named `2022-letter.txt`. You can update this path to match the location of your document.

- **`with open(letter_path, "r") as file:`**:  
  This line opens the file at the specified `letter_path` in **read mode** (`"r"`). The **`with`** statement ensures that the file is automatically closed after the operations are completed.

- **`letter = file.read()`**:  
  This line reads the entire content of the file and stores it in the variable `letter`. The contents of the file are now stored as a string, and we can process this string for summarization.

### Why Load the Document?

- **Processing Text**: We load the document so that we can process its content and generate summaries or perform other text-based tasks.

- **Simplification**: By loading the entire document into memory, we simplify the process of passing it to the model for summarization, making the task more straightforward.



In [3]:
letter_path = "./letters/2022-letter.txt"  # Update path if needed

with open(letter_path, "r") as file:
    letter = file.read()


## Step 5: Split the Document into Chunks

To avoid exceeding the model's token limit, we will split the document into smaller, manageable chunks. This helps ensure that the model can process the content efficiently.

### Why This Code?

This code splits the document into chunks of text to ensure that each chunk fits within the model### Explanation of the Code:

- **`from langchain_text_splitters import RecursiveCharacterTextSplitter`**:  
  This line imports the **`RecursiveCharacterTextSplitter`** class from the **`langchain_text_splitters`** module. This class is used to split the document into smaller chunks based on a specified character length, making it easier for the model to process.

- **`text_splitter = RecursiveCharacterTextSplitter(...)`**:  
  This line creates an instance of the `RecursiveCharacterTextSplitter` class, initializing it with specific parameters:
  - **`separators=["\n\n", "\n"]`**: This defines the delimiters used to split the document. The text will be split wherever there are double newlines (`\n\n`) or single newlines (`\n`).
  - **`chunk_size=4000`**: This parameter specifies the **maximum size** of each chunk in terms of characters. The document will be split into chunks of up to **4000 characters**.
  - **`chunk_overlap=100`**: This specifies the **overlap** between consecutive chunks. It ensures that each chunk overlaps with the previous one by **100 characters**, which helps maintain context between chunks.

- **`docs = text_splitter.create_documents([letter])`**:  
  This line processes the document stored in `letter` and splits it into smaller chunks based on the settings defined in the previous step. The result is stored in the `docs` list.

- **`print(f"{len(docs)} chunks loaded")`**:  
  This line prints the number of chunks the document has been split into, which helps to confirm how many pieces the model will process.

### Why Split the Document?

- **Token Limits**: Large documents may exceed the **token limits** of the model. By splitting the document into smaller chunks, we ensure that each chunk fits within the model’s processing capacity.
  
- **Maintaining Context**: The overlap ensures that important context is retained between chunks, making the summary more coherent.

- **Simplifying Processing**: Breaking the document into smaller, manageable parts simplifies the process of passing it to the model for summarization or other NLP tasks.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],
    chunk_size=4000,
    chunk_overlap=100
)

docs = text_splitter.create_documents([letter])
print(f"{len(docs)} chunks loaded")


3 chunks loaded


## Step 6: Define the Summarization Prompts

In this step, we define the prompts that will guide the model to generate and refine the summary. We first create an **initial summary** and then **refine** it using additional context.

### Why This Code?

This code defines two prompts: one for the **initial summarization** and one for **refining the summary**. The **refining** process helps make the summary more accurate and comprehensive by adding more context from subsequent chunks of the document.

### Explanation of the Code:

- **`from langchain_core.prompts import PromptTemplate`**:  
  This imports the **`PromptTemplate`** class, which allows us to create a template for the prompt that will be passed to the model. A **prompt template** is a pre-defined string that can be filled dynamically with content. This helps us structure the prompt to be used by the model.

- **`initial_prompt = PromptTemplate.from_template(...)`**:  
  This line creates the **initial summary prompt template**. The prompt asks the model to write a concise summary of the provided document. The **`{context}`** placeholder will be dynamically replaced with the content of the document. The model will use this prompt to generate the first summary of the document.

- **`refine_prompt = PromptTemplate.from_template(...)`**:  
  This defines the **refining summary prompt**. It asks the model to refine an existing summary by adding additional context. The **`{existing_answer}`** placeholder will be replaced by the previously generated summary, and the **`{context}`** placeholder will be filled with new content from the document. This helps the model improve the initial summary by incorporating more context.

- **`def refine_chain(docs):`**:  
  This function defines how the **refining process** works. It starts by generating the initial summary using the first chunk of the document. Then, it loops through the remaining chunks of the document, refining the summary by adding more context with each chunk. The refined summary is updated iteratively until all chunks are processed.

- **`summary_chain = RunnableLambda(refine_chain)`**:  
  This creates a **`RunnableLambda`** object, which is a runnable chain that allows us to execute the **`refine_chain`** function. This function will process all the document chunks and generate the final refined summary, which incorporates all context from the document.


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

initial_prompt = PromptTemplate.from_template(
    "Write a concise summary of the following document:\n\n{context}"
)

refine_prompt = PromptTemplate.from_template(
    "We have an existing summary:\n\n{existing_answer}\n\n"
    "Refine it using the following additional context:\n\n{context}\n\n"
    "Return a new, improved summary:"
)

def refine_chain(docs):
    summary = llm.invoke(initial_prompt.format(context=docs[0].page_content))
    for doc in docs[1:]:
        summary = llm.invoke(refine_prompt.format(existing_answer=summary, context=doc.page_content))
    return summary

summary_chain = RunnableLambda(refine_chain)


## Step 7: Generate the Summary

In this step, we invoke the **`summary_chain`** to generate the summary by processing the document chunks. If any errors occur during this process, they will be caught and handled.

### Why This Code?

This code executes the **`summary_chain`** function to generate the summary. It handles errors gracefully, ensuring that if the input is not formatted correctly, an appropriate message is shown.

### Explanation of the Code:

- **`output = ""`**:  
  This line initializes the `output` variable as an empty string. This variable will store the generated summary once the **`summary_chain`** function is invoked.

- **`try:`**:  
  The **`try`** block is used to attempt the execution of the **`summary_chain.invoke(docs)`** function. This function processes the document chunks and generates the summary. If it executes successfully, the result is stored in the `output` variable.

- **`output = summary_chain.invoke(docs)`**:  
  This line calls the **`summary_chain.invoke(docs)`** method to generate the summary by processing the document chunks stored in **`docs`**. The generated summary is stored in the `output` variable.

- **`except ValueError as error:`**:  
  If a **`ValueError`** is raised (e.g., due to an issue with the input format), the code inside the **`except`** block will be executed.

- **`if "ValidationException" in str(error):`**:  
  This checks whether the error message contains **`ValidationException`**, which indicates that the input is not properly formatted. If the error matches this condition, the following message is printed:
  
  - **`print(f"Validation Error: {error}. Ensure input is formatted correctly.")`**: This prints the error message along with a suggestion to check the input format.

- **`else: raise error`**:  
  If the error is not related to input validation, the **`else`** block raises the error again, so it can be handled elsewhere or stop the program execution.

- **`print(output.strip())`**:  
  Finally, this line prints the generated summary. The **`strip()`** method is used to remove any leading or trailing whitespace from the summary.

### Why Handle Errors?

- **Graceful Error Handling**: This ensures that if the input format is wrong, the program will provide a helpful message rather than crashing.

- **Input Validation**: If there's an issue with the formatting or input, handling errors helps identify the problem and makes debugging easier.

- **Ensuring Clean Output**: The **`strip()`** method ensures that the output summary does not contain unnecessary whitespace, making it cleaner and more readable.


In [6]:
output = ""
try:
    output = summary_chain.invoke(docs)
except ValueError as error:
    if "ValidationException" in str(error):
        print(f"Validation Error: {error}. Ensure input is formatted correctly.")
    else:
        raise error

print(output.strip())


Amazon CEO Dave Clark discusses the company's progress in 2022, including growth, innovation, and investment decisions. He highlights the company's commitment to customer experience, third-party marketplace, and long-term investments. Clark also mentions the challenges faced by the company in 2022, including macroeconomic conditions. Despite these challenges, Amazon was able to grow demand and innovate in its largest businesses. The company also made adjustments in its investment decisions and the way it will invent moving forward, while preserving long-term investments. Clark emphasizes the constant change in the global market and the need for companies to adapt and innovate to stay competitive. He concludes by expressing optimism and energy for the future of Amazon.

There have also been times when macroeconomic conditions or operating inefficiencies have presented us with new challenges. For instance, in the 2001 dot-com crash, we had to secure letters of credit to buy inventory for

## What You Learned

In this lab, you learned how to use **Amazon Titan** and **LangChain** to perform **abstractive text summarization** on a document. You went through the following key steps:

1. **Setting up the AWS environment**:  
   You learned how to configure AWS environment variables such as the **AWS region** and **credentials** to ensure smooth communication with Amazon services.

2. **Loading the document**:  
   You learned how to load the document that needs to be summarized. The document is read into memory, making it ready for processing by the model.

3. **Splitting the document into chunks**:  
   You learned how to split large documents into smaller chunks to avoid exceeding the model’s token limit, ensuring efficient processing by the model.

4. **Creating summarization prompts**:  
   You learned how to define the **initial summary prompt** to summarize the first chunk and a **refining prompt** to improve the summary with each subsequent chunk of the document.

5. **Generating the summary**:  
   You learned how to invoke the **summary chain** to generate the final summary. You also handled errors gracefully to ensure the process runs smoothly.

### Key Takeaways:

- **Text summarization** with **Amazon Titan** and **LangChain** enables efficient summarization of large text datasets.
- **Token limits** can be managed by splitting documents into chunks, ensuring that each chunk fits within the model’s processing capacity.
- The **refining process** improves the coherence and accuracy of the summary by adding context as more chunks are processed.
- **Error handling** ensures the process runs without interruptions and that issues with input format are managed appropriately.

By following these steps, you gained practical experience in working with **state-of-the-art language models** and **cloud services** to automate text summarization.
